- **Examples:** https://docs.manim.community/en/stable/examples.html


Few Summary From Code Snippet:
- Always extend from ``ThreeDScene`` for constructing 3d animations. It supports 3d camera movement.


## ThreeDSceneAttributes:
- ``phi`` => The polar angle i.e the angle between Z_AXIS and Camera **through ORIGIN** in **radians**. ( use * DEGREE for degrees)
- ``theta`` => The azimuthal angle i.e the **angle that spins the camera around the Z_AXIS** in **radians** (must use * DEGREE for degrees)



In [ ]:
# plane_explainer_IMPROVED.py
# Manim Community v0.19.0
#
# Run:
#    manim -pql plane_explainer_IMPROVED.py ThreeDPlaneExplainer

from manim import *
import numpy as np

# -------------------------
# Color constants
# -------------------------
VAR_COLORS = {
    "a": RED, "b": BLUE, "c": GREEN, "d": ORANGE,
    "l": MAROON, "m": DARK_BLUE, "n": TEAL, "p": PURPLE,  # 'm' is now DARK_BLUE (was YELLOW_D)
    "x": DARK_BROWN, "y": DARK_BROWN, "z": DARK_BROWN,   # x,y,z are DARK_BROWN (was GOLD)
    "v": PINK,                                          # 'v' is PINK (was GOLD)
    "P": PINK, "Q": BLACK,                              # 'P' is PINK (was GOLD)
}

def colored_math(tex: str, var_colors=VAR_COLORS, **kwargs) -> MathTex:
    """Create MathTex and color variable tokens that appear."""
    m = MathTex(rf"{tex}", **kwargs)
    for var, color in var_colors.items():
        try:
            m.set_color_by_tex(var, color)
            m.font_size = kwargs.get("font_size", 36)
        except Exception:
            pass
    return m

class ThreeDPlaneExplainer(ThreeDScene):
    """
    Six-step 3D explainer.
    Steps and method names must match exactly.
    """

    def construct(self):
        # camera
        self.set_camera_orientation(phi=75 * DEGREES, theta=-45 * DEGREES)
        self.camera.background_color = BLACK

        # IMPORTANT: method names must exist exactly as listed here
        steps = [
            self.step_standard_form,
            # self.step_intercept_form,
            # self.step_normal_form,
            # self.step_angle_between_planes,
            # self.step_angle_line_plane,
            # self.step_point_plane_distance,
        ]

        for idx, fn in enumerate(steps, start=1):
            title = Text(f"Concept {idx}: {fn.__doc__.strip()}", font_size=28, color=WHITE)
            title.to_edge(UP)
            self.add_fixed_in_frame_mobjects(title)
            self.play(FadeIn(title), run_time=0.35)

            # call the step (each step is a method defined below)
            fn()

            self.wait(1.5)

            # clear 3D and fixed (safe checks)
            if len(self.mobjects) > 0:
                self.play(FadeOut(*self.mobjects), run_time=0.6)
            if len(self.foreground_mobjects) > 0:
                self.play(FadeOut(*self.foreground_mobjects), run_time=0.4)
            self.clear()
            self.wait(0.12)

    # ---------- helpers ----------
    def make_axes(self, size=6):
        axes = ThreeDAxes(x_length=size, y_length=size, z_length=size)
        axes.set_stroke(width=1, opacity=1, color=WHITE)
        return axes

    def left_position(self, *mobjects):
        g = VGroup(*mobjects)
        g.to_edge(LEFT)
        g.shift(RIGHT * 0.5)
        return g

    def right_panel_fixed(self, *items):
        panel = VGroup(*items)
        panel.arrange(DOWN, aligned_edge=LEFT, buff=0.45)
        panel.scale(0.95)
        panel.to_edge(RIGHT)
        panel.shift(LEFT * 0.25)
        for child in panel:
            if isinstance(child, (Text, MathTex)):
                child.set_color(WHITE)
        self.add_fixed_in_frame_mobjects(panel)
        return panel

    # -------------------------
    # Step 1
    # -------------------------
    def step_standard_form(self):
        r"""Equation: ax+by+cz+d=0 (Standard Form)"""
        axes = self.make_axes()
        a, b, c, d = 1.0, 1.5, -1.0, -2.0

        # --- Define the plane surface ---
        plane = Surface(
            lambda u, v: np.array([u, v, (-a * u - b * v - d) / c]),
            u_range=[-2, 2], v_range=[-2, 2], resolution=(20, 20)
        )
        plane.set_fill(GREEN, opacity=0.6).set_stroke(width=0.5)

        self.play(Create(axes), FadeIn(plane))

        # --- Normal vector ---
        normal = np.array([a, b, c])
        normal = normal / np.linalg.norm(normal)  # normalize

        # --- Arrow showing normal direction ---
        normal_arrow = Arrow(start=ORIGIN, end=2.5 * normal, buff=0)
        normal_arrow.set_stroke(width=4).set_color(ORANGE)

        # --- Add extra visuals (left panel, etc.) ---
        left = self.left_position(axes, plane, normal_arrow)
        self.add(left)

        # --- Compute center and camera position ---
        plane_center = plane.get_center()
        camera_distance = 5
        camera_position = plane_center + camera_distance * normal

        # --- Initial camera setup ---
        self.move_camera(
            frame_center=plane_center,
            phi=70 * DEGREES,
            theta=-45 * DEGREES
        )

        # --- Right side info panel ---
        eq = colored_math(r"ax + by + cz + d = 0", font_size=46)
        normal_math = colored_math(r"\vec{n} = \langle a, b, c \rangle", font_size=26)
        desc1 = Text("a, b, c are components of the plane's normal vector", font_size=20, color=WHITE)
        desc2 = Text("d is the signed distance (offset) from origin", font_size=20, color=WHITE)
        panel = self.right_panel_fixed(Text("Standard Form", font_size=26), eq, normal_math, desc1, desc2)

        # --- Fixed label near the normal arrow tip ---
        tip = normal_arrow.get_end()
        normal_label = colored_math(r"\vec{n}", font_size=28, color=ORANGE)
        self.add_fixed_in_frame_mobjects(normal_label)
        normal_label.move_to(self.camera.project_point(tip))
        normal_label.shift(RIGHT * 0.4 + UP * 0.2)

        # --- Animation: Draw the normal and show text ---
        self.play(GrowArrow(normal_arrow))
        self.play(Write(panel), Write(normal_label))
        self.wait(1)

        # --- Show the dot product form: n·r + d = 0 ---
        dot_form = colored_math(r"\vec{n} \cdot \vec{r} + d = 0", font_size=38)
        dot_desc = Text("Geometric form using dot product", font_size=20, color=WHITE)
        self.play(FadeIn(dot_form.next_to(panel, DOWN)), FadeIn(dot_desc.next_to(dot_form, DOWN, buff=0.2)))
        self.wait(1.5)

        # --- Animate camera moving toward the normal direction ---
        # Smoothly move along the normal vector and stop facing the plane
        self.play(
            self.camera.frame.animate.move_to(camera_position).look_at(plane_center),
            run_time=4,
            rate_func=smooth
        )
        self.wait(2)

        # Optional: fade out highlight
        self.play(FadeOut(dot_form), FadeOut(dot_desc))
        self.wait(1)


    # -------------------------
    # Step 2
    # -------------------------
    # def step_intercept_form(self):
    #     r"""Equation: $\frac{x}{a}+\frac{y}{b}+\frac{z}{c}=1$ (Intercept Form)"""
    #     axes = self.make_axes()
    #     A, B, C = 3.0, 2.0, -4.0
    #     pA, pB, pC = np.array([A, 0, 0]), np.array([0, B, 0]), np.array([0, 0, C])

    #     tri = Polygon(pA, pB, pC)
    #     tri.set_fill(BLUE_E, opacity=0.7).set_stroke(width=0.5)
    #     sA = Sphere(radius=0.07).move_to(pA).set_color(VAR_COLORS["a"])
    #     sB = Sphere(radius=0.07).move_to(pB).set_color(VAR_COLORS["b"])
    #     sC = Sphere(radius=0.07).move_to(pC).set_color(VAR_COLORS["c"])

    #     left = self.left_position(axes, tri, sA, sB, sC)
    #     self.add(left)

    #     eq = colored_math(r"\frac{x}{a}+\frac{y}{b}+\frac{z}{c}=1", font_size=42)
    #     desc = Text("a,b,c are intercepts on x,y,z axes respectively", font_size=20)
    #     panel = self.right_panel_fixed(Text("Intercept Form", font_size=26), eq, desc)

    #     # IMPROVED: Use colored_math and add shifts
    #     la = colored_math(r"(a,0,0)", font_size=18)
    #     lb = colored_math(r"(0,b,0)", font_size=18)
    #     lc = colored_math(r"(0,0,c)", font_size=18)
    #     self.add_fixed_in_frame_mobjects(la, lb, lc)
    #     la.move_to(axes.c2p(*self.camera.project_point(pA)[:2], 0)).shift(RIGHT*0.4 + UP*0.1)
    #     lb.move_to(axes.c2p(*self.camera.project_point(pB)[:2], 0)).shift(LEFT*0.4 + UP*0.1)
    #     lc.move_to(axes.c2p(*self.camera.project_point(pC)[:2], 0)).shift(DOWN*0.3)

    #     self.play(Create(axes), FadeIn(tri))
    #     self.play(FadeIn(sA), FadeIn(sB), FadeIn(sC), Write(la), Write(lb), Write(lc))
    #     self.play(Write(panel))

    # -------------------------
    # Step 3
    # -------------------------
    # def step_normal_form(self):
    #     r"""Equation: $lx+my+nz=p$ (Normal Form)"""
    #     axes = self.make_axes()

    #     l, m, n = 1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)
    #     p_val = 2.0
    #     normal_vec = np.array([l, m, n])
    #     center = p_val * normal_vec

    #     def plane_param(u, v):
    #         w1 = np.array([m, -l, 0.0])
    #         if np.linalg.norm(w1) < 1e-6:
    #             w1 = np.array([0.0, n, -m])
    #         w1 /= np.linalg.norm(w1)
    #         w2 = np.cross(normal_vec, w1)
    #         return center + 1.5 * (u - 0.5) * w1 + 1.5 * (v - 0.5) * w2

    #     plane = Surface(plane_param, u_range=[0, 1], v_range=[0, 1], resolution=(15, 15))
    #     plane.set_fill(PURPLE_A, opacity=0.6).set_stroke(width=0.5)
    #     arrow = Arrow(start=ORIGIN, end=2.5 * normal_vec).set_stroke(width=4).set_color(PURPLE)
    #     dline = Line3D(start=ORIGIN, end=center).set_stroke(width=3).set_color(PURPLE)

    #     left = self.left_position(axes, plane, arrow, dline)
    #     self.add(left)

    #     eq = colored_math(r"l x + m y + n z = p", font_size=44)
    #     unit = colored_math(r"l^{2}+m^{2}+n^{2}=1", font_size=22) # IMPROVED
    #     desc = Text("Unit normal and distance p from origin", font_size=20)
    #     panel = self.right_panel_fixed(Text("Normal Form", font_size=26), eq, unit, desc)

    #     # IMPROVED: Use colored_math and add shift
    #     p_label = colored_math("p", font_size=20)
    #     self.add_fixed_in_frame_mobjects(p_label)
    #     p_label.move_to(axes.c2p(*self.camera.project_point(center / 2.0)[:2], 0)) # Move to midpoint
    #     p_label.shift(RIGHT*0.2)

    #     self.play(Create(axes), FadeIn(plane))
    #     self.play(GrowArrow(arrow), Create(dline))
    #     self.play(Write(panel), Write(p_label))

    # -------------------------
    # Step 4
    # -------------------------
    # def step_angle_between_planes(self):
    #     r"""Formula: angle between planes via normals (cosθ = (n1·n2)/|n1||n2|)"""
    #     axes = self.make_axes()

    #     n1 = np.array([1.0, 1.0, 0.5])
    #     n2 = np.array([0.5, -1.0, 1.0])

    #     def make_plane(normal, shift, color):
    #         nu = normal / np.linalg.norm(normal)
    #         b1 = np.cross(nu, UP if np.linalg.norm(np.cross(nu, UP)) > 1e-6 else RIGHT)
    #         b1 /= np.linalg.norm(b1)
    #         b2 = np.cross(nu, b1)
    #         return Surface(lambda u, v: shift + 1.5*(u-0.5)*b1 + 1.5*(v-0.5)*b2,
    #                         u_range=[0, 1], v_range=[0, 1], resolution=(15, 15)).set_fill(color, opacity=0.6).set_stroke(width=0.5)

    #     p1 = make_plane(n1, np.array([-1.5, 0, 0]), RED_A)
    #     p2 = make_plane(n2, np.array([1.5, 0, 0]), BLUE_A)
    #     a1 = Arrow(start=ORIGIN, end=2.0 * n1 / np.linalg.norm(n1)).set_color(RED)
    #     a2 = Arrow(start=ORIGIN, end=2.0 * n2 / np.linalg.norm(n2)).set_color(BLUE)

    #     left = self.left_position(axes, p1, p2, a1, a2)
    #     self.add(left)

    #     # IMPROVED: Manually color n_1 and n_2
    #     eq = MathTex(r"\cos\theta=\frac{\vec{n}_1\cdot\vec{n}_2}{\|\vec{n}_1\|\|\vec{n}_2\|}", font_size=34)
    #     eq.set_color_by_tex("n_1", RED)
    #     eq.set_color_by_tex("n_2", BLUE)
        
    #     dot = float(np.dot(n1, n2))
    #     mag = float(np.linalg.norm(n1) * np.linalg.norm(n2))
    #     theta = np.degrees(np.arccos(np.clip(dot / mag, -1.0, 1.0)))
    #     numeric = MathTex(rf"\theta = {theta:.1f}^\circ", font_size=28)
    #     panel = self.right_panel_fixed(Text("Angle Between Planes", font_size=26), eq, numeric)

    #     # IMPROVED: Use MathTex, colors, and shifts
    #     n1_label = MathTex(r"\vec{n}_1", font_size=18, color=RED)
    #     n2_label = MathTex(r"\vec{n}_2", font_size=18, color=BLUE)
    #     self.add_fixed_in_frame_mobjects(n1_label, n2_label)
    #     n1_label.move_to(axes.c2p(*self.camera.project_point(a1.get_end())[:2], 0)).shift(UP*0.2)
    #     n2_label.move_to(axes.c2p(*self.camera.project_point(a2.get_end())[:2], 0)).shift(RIGHT*0.2)

    #     self.play(Create(axes), FadeIn(p1), FadeIn(p2))
    #     self.play(GrowArrow(a1), GrowArrow(a2), Write(n1_label), Write(n2_label))
    #     self.play(Write(panel))

    # -------------------------
    # Step 5
    # -------------------------
    # def step_angle_line_plane(self):
    #     r"""Formula: angle between line & plane = 90° - φ ; cosφ = |v·n|/(|v||n|)"""
    #     axes = self.make_axes()

    #     v = np.array([2.0, 1.0, 3.0])
    #     n = np.array([1.0, -1.0, 1.0])

    #     line = Line3D(start=-2*v, end=2*v).set_stroke(width=3).set_color(VAR_COLORS["v"])
    #     plane = Surface(lambda u, vv: np.array([u, vv, (-n[0]*u - n[1]*vv)/n[2]]),
    #                     u_range=[-3, 3], v_range=[-3, 3], resolution=(18, 18))
    #     plane.set_fill(GREY_B, opacity=0.6).set_stroke(width=0.5)
    #     v_arr = Arrow(start=ORIGIN, end=2.5 * v / np.linalg.norm(v)).set_color(VAR_COLORS["v"])
    #     n_arr = Arrow(start=ORIGIN, end=2.5 * n / np.linalg.norm(n)).set_color(VAR_COLORS["n"])

    #     left = self.left_position(axes, plane, line, v_arr, n_arr)
    #     self.add(left)

    #     # IMPROVED: Use colored_math
    #     eq_phi = colored_math(r"\cos\phi=\frac{|\vec{v}\cdot\vec{n}|}{\|\vec{v}\|\|\vec{n}\|}", font_size=30)
    #     dot = abs(float(np.dot(v, n)))
    #     denom = float(np.linalg.norm(v) * np.linalg.norm(n))
    #     phi_deg = np.degrees(np.arccos(np.clip(dot / denom, -1.0, 1.0)))
    #     angle_lp = 90.0 - phi_deg
    #     eq_angle = MathTex(rf"\text{{Angle (line, plane)}} = 90^\circ - \phi = {angle_lp:.1f}^\circ", font_size=26)
    #     panel = self.right_panel_fixed(Text("Angle between Line & Plane", font_size=26), eq_phi, eq_angle)

    #     # IMPROVED: Use colored_math and shifts
    #     v_label = colored_math(r"\vec{v}", font_size=18)
    #     n_label = colored_math(r"\vec{n}", font_size=18)
    #     self.add_fixed_in_frame_mobjects(v_label, n_label)
    #     v_label.move_to(axes.c2p(*self.camera.project_point(v_arr.get_end())[:2], 0)).shift(UP*0.2)
    #     n_label.move_to(axes.c2p(*self.camera.project_point(n_arr.get_end())[:2], 0)).shift(RIGHT*0.2)

    #     self.play(Create(axes), FadeIn(plane), Create(line))
    #     self.play(GrowArrow(v_arr), GrowArrow(n_arr), Write(v_label), Write(n_label))
    #     self.play(Write(panel))

    # # -------------------------
    # # Step 6
    # # -------------------------
    # def step_point_plane_distance(self):
    #     r"""Formula: Distance = |ax+by+cz+d| / sqrt(a^2+b^2+c^2)"""
    #     axes = self.make_axes()

    #     a, b, c, d = 2.0, -1.0, 1.0, -3.0
    #     P = np.array([2.5, 1.0, 0.5]) # Point P

    #     plane = Surface(lambda u, v: np.array([u, v, (-a * u - b * v - d) / c]),
    #                     u_range=[-3, 3], v_range=[-3, 3], resolution=(22, 22))
    #     plane.set_fill(BLUE_E, opacity=0.6).set_stroke(width=0.5)

    #     normal = np.array([a, b, c])
    #     numerator = float(np.dot(normal, P) + d)
    #     denom = np.linalg.norm(normal)
    #     distance = abs(numerator) / denom
    #     Q = P - (numerator / (denom ** 2)) * normal # Point Q (projection)

    #     P_s = Sphere(radius=0.08).move_to(P).set_color(VAR_COLORS["P"])
    #     Q_s = Sphere(radius=0.06).move_to(Q).set_color(VAR_COLORS["Q"])
    #     perp = Line3D(start=P, end=Q).set_stroke(width=3).set_color(RED)

    #     left = self.left_position(axes, plane, P_s, Q_s, perp)
    #     self.add(left)

    #     # IMPROVED: Use colored_math
    #     eq = colored_math(r"\mathrm{Distance}=\frac{|ax_P+by_P+cz_P+d|}{\sqrt{a^{2}+b^{2}+c^{2}}}", font_size=32)
    #     eq.set_color_by_tex("x_P", VAR_COLORS["P"]) # Color point coords
    #     eq.set_color_by_tex("y_P", VAR_COLORS["P"])
    #     eq.set_color_by_tex("z_P", VAR_COLORS["P"])
        
    #     numeric = MathTex(rf"= {distance:.3f}", font_size=26)
    #     panel = self.right_panel_fixed(Text("Point-to-Plane Distance", font_size=26), eq, numeric)

    #     # IMPROVED: Use colored_math and shifts
    #     labP = colored_math("P", font_size=18)
    #     labQ = colored_math("Q", font_size=18)
    #     self.add_fixed_in_frame_mobjects(labP, labQ)
    #     labP.move_to(axes.c2p(*self.camera.project_point(P)[:2], 0)).shift(UP*0.2)
    #     labQ.move_to(axes.c2p(*self.camera.project_point(Q)[:2], 0)).shift(DOWN*0.2)

    #     self.play(Create(axes), FadeIn(plane))
    #     self.play(FadeIn(P_s), Write(labP))
    #     self.play(FadeIn(Q_s), Create(perp), Write(labQ))
    #     self.play(Write(panel))

%manim -ql -v warning ThreeDPlaneExplainer

Manim Community v0.19.0

AttributeError: 'ThreeDCamera' object has no attribute 'frame'